In [1]:
import pandas as pd
import numpy as np

In [2]:
cost_warehouse = 50000000
mile_cost = 5
km_cost = mile_cost * 1.6
warehouse_cap = 1000000
truck_cap = 40000

In [3]:
dist = pd.read_csv('dist.csv')
dist = dist.pivot(index='Node 1', columns='Node 2', values='Distance (KM)').fillna(0)

In [4]:
cost = (dist.astype(int) * km_cost).astype(int)

In [5]:
dist

Node 2,Basel,Bern,Biel,Chur,Fribourg,Geneva,Lausanne,Lucerne,Lugano,Montreux,Saint Gallen,Schaffhausen,Sion,Thun,Winterthur,Zug,Zurich
Node 1,,,,,,,,,,,,,,,,,
Basel,0,111,90,204,130,251,187,95,266,182,164,130,249,122,106,108,85
Bern,111,0,53,248,37,161,105,116,293,90,217,175,156,27,159,138,130
Biel,90,53,0,237,80,161,109,105,274,132,198,163,200,72,140,129,117
Chur,204,248,237,0,277,399,343,143,145,330,106,185,396,270,159,117,121
Fribourg,130,37,80,277,0,140,72,145,312,60,237,203,130,69,183,167,159
Geneva,251,161,161,399,140,0,71,266,370,95,360,325,163,190,301,290,282
Lausanne,187,105,109,343,72,71,0,209,380,29,304,269,95,137,246,233,225
Lucerne,95,116,105,143,145,266,209,0,171,196,143,101,267,97,77,31,51
Lugano,266,293,274,145,312,370,380,171,0,278,249,256,217,248,240,172,208


In [6]:
population = pd.read_csv('csvData.csv')

In [7]:
population

,2022,name
0,341730,Zurich
1,183981,Geneve
2,164488,Basel
3,121631,Bern
4,116751,Lausanne
...,...,...
338,5061,Estavayer-le-Lac
339,5045,Sargans
340,5032,Greifensee
341,5031,Wallisellen / Wallisellen-Ost


In [8]:
cities = pd.Series(cost.index).str.replace('\xa0','')

In [9]:
cities

0            Basel
1             Bern
2             Biel
3             Chur
4         Fribourg
5           Geneva
6         Lausanne
7          Lucerne
8           Lugano
9         Montreux
10    Saint Gallen
11    Schaffhausen
12            Sion
13            Thun
14      Winterthur
15             Zug
16          Zurich
Name: Node 1, dtype: object

In [10]:
relevant_pop = population[population['name'].isin(cities)]

In [11]:
additional_cities = ['Geneve', 'Luzern', 'Sankt Gallen', 'Biel/Bienne']
relevant_pop = pd.concat((relevant_pop, population.loc[population['name'].isin(additional_cities)]))
sion = {'2022': 32797, 'name':'Sion'}
relevant_pop = pd.concat((relevant_pop, pd.DataFrame([sion])))

In [12]:
relevant_pop = relevant_pop.sort_values('name')
relevant_pop

,2022,name
2,164488,Basel
3,121631,Bern
10,48614,Biel/Bienne
21,32429,Chur
20,32827,Fribourg
1,183981,Geneve
4,116751,Lausanne
7,63000,Lugano
8,57066,Luzern
36,22897,Montreux


In [13]:
demand = np.array(relevant_pop['2022'])
demand

array([164488, 121631,  48614,  32429,  32827, 183981, 116751,  63000,
        57066,  22897,  70572,  33863,  32797,  42136,  91908,  23435,
       341730], dtype=int64)

In [14]:
distance = np.array(dist.astype(int))

In [15]:
# p = []
# for i in range(4):
#     p.append([])
#     for j in range(4):
#         p[i].append(1)
# p

In [16]:
#[p[i][j] * distance[i][j] * km_cost for i in range(4) for j in range(4)]

In [17]:
cost_warehouse = 50000000
mile_cost = 5
km_cost = mile_cost * 1.6
warehouse_cap = 1000000
truck_cap = 40000

In [32]:
from ortools.sat.python import cp_model as cp

model = cp.CpModel()

# data
max_distance = 100
num_cities = len(cities)
    
# declare variables
x = []
c = []
for i in range(num_cities):
    x.append(model.NewBoolVar("x[%i]" % i))
    c.append([])
    for j in range(num_cities):
        c[i].append(model.NewIntVar(0, 400000, "c[%i][%i]" % (i, j)))

z = model.NewIntVar(0, 999999999999, "z")

# objective to minimize
model.Add(z == (sum(x) * cost_warehouse + sum([c[i][j] * distance[i][j] for i in range(num_cities) for j in range(num_cities)])))
# sum(x) * cost_warehouse + sum([c[i][j] * distance[i][j] * km_cost for i in range(num_cities) for j in range(num_cities)])

# constraints
for j in range(num_cities):
    model.Add(sum([c[i][j] for i in range(num_cities)]) >= demand[j])
    
for i in range(num_cities):
    model.Add(sum([c[i][j] for j in range(num_cities)]) <= 400000*x[i])
    
for i in range(num_cities):
    model.Add(sum([c[i][j] for j in range(num_cities)]) >= 0)

model.Minimize(z)

# solution and search
solver = cp.CpSolver()
status = solver.Solve(model)

if status == cp.OPTIMAL:
    print("z:", solver.Value(z))
    print("x:", [solver.Value(x[i]) for i in range(num_cities)])
    print("c:", [solver.Value(c[i][j]) for i in range(num_cities) for j in range(num_cities)])

print("NumConflicts:", solver.NumConflicts())
print("NumBranches:", solver.NumBranches())

z: 266267435
x: [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]
c: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 78491, 121631, 48614, 0, 32827, 0, 0, 0, 0, 22897, 0, 0, 32797, 42136, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 183981, 116751, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 85997, 0, 0, 0, 0, 0, 0, 63000, 0, 0, 70572, 33863, 0, 0, 91908, 0, 54660, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32429, 0, 0, 0, 0, 57066, 0, 0, 0, 0, 0,

In [33]:
warehouses = np.array([solver.Value(x[i]) for i in range(num_cities)])
cities[warehouses > 0]

1           Bern
5         Geneva
14    Winterthur
16        Zurich
Name: Node 1, dtype: object

In [35]:
sent = np.array([solver.Value(c[i][j]) for i in range(num_cities) for j in range(num_cities)])
for i, val in enumerate(sent):
    if val > 0:
        from_i = i // num_cities
        to_i = i% num_cities
        print('from:',cities[from_i],'to:',cities[to_i], 'amount:', val, 'distance:', distance[from_i][to_i], 'price:', distance[from_i][to_i] * val)

from: Bern to: Basel amount: 78491 distance: 111 price: 8712501
from: Bern to: Bern amount: 121631 distance: 0 price: 0
from: Bern to: Biel amount: 48614 distance: 53 price: 2576542
from: Bern to: Fribourg amount: 32827 distance: 37 price: 1214599
from: Bern to: Montreux amount: 22897 distance: 90 price: 2060730
from: Bern to: Sion amount: 32797 distance: 156 price: 5116332
from: Bern to: Thun amount: 42136 distance: 27 price: 1137672
from: Geneva to: Geneva amount: 183981 distance: 0 price: 0
from: Geneva to: Lausanne amount: 116751 distance: 71 price: 8289321
from: Winterthur to: Basel amount: 85997 distance: 106 price: 9115682
from: Winterthur to: Lucerne amount: 63000 distance: 77 price: 4851000
from: Winterthur to: Saint Gallen amount: 70572 distance: 56 price: 3952032
from: Winterthur to: Schaffhausen amount: 33863 distance: 34 price: 1151342
from: Winterthur to: Winterthur amount: 91908 distance: 0 price: 0
from: Winterthur to: Zurich amount: 54660 distance: 27 price: 1475820
fr